In [1]:
import pandas as pd
from tqdm.contrib.concurrent import process_map

from notebooks.ford.asbuilt import AsBuiltData
from notebooks.ford.decode import search, print_breakdown


df_nhtsa = await search(
  min_model_year=2017,
  include_openpilot=True,
  include_police=True,
  skip_missing_asbuilt=True,
)

# pre-load asbuilt
process_map(AsBuiltData.from_vin, df_nhtsa['VIN'].unique(), desc='Loading AsBuilt Data', chunksize=100)

print()
print_breakdown(df_nhtsa, include_model_year=False)

Loaded 36764 VINs (filter_comment=None, include_openpilot=True, skipped=163, missing_asbuilt=0)


Loading AsBuilt Data:   0%|          | 0/36762 [00:00<?, ?it/s]


Model
                      2
Aviator            1107
Bronco              996
Bronco Sport       1500
C-Max                 5
Continental          23
Corsair             986
Ecosport            406
Edge               2685
Escape             3568
Expedition          835
Expedition MAX      733
Explorer           3364
F-150              8253
F-150 Lightning     385
F-250              1720
F-350              1158
F-450               207
F-550                 1
Fiesta              195
Flex                189
Focus               169
Fusion             1163
GT                    3
MKC                  55
MKT                   7
MKZ                 100
Maverick            994
Mustang             952
Mustang Mach-E      839
Nautilus           1169
Navigator           303
Navigator L         230
Ranger              533
Taurus              122
Transit            1391
Transit Connect     414
dtype: int64


In [2]:
from notebooks.ford.ecu import FordEcu
from panda.python.uds import DATA_IDENTIFIER_TYPE


def get_platform_code(row):
  abd = AsBuiltData.from_vin(row['VIN'])

  eps_fw = abd.get_identifier(FordEcu.PowerSteeringControlModule, DATA_IDENTIFIER_TYPE.VEHICLE_MANUFACTURER_ECU_SOFTWARE_NUMBER)
  if eps_fw is None:
    platform_code = None
  else:
    prefix, core, _ = eps_fw.split('-')
    platform_code = f'{core}-{prefix[1:]}'

  return pd.Series({
    'PlatformCode': platform_code,
    'PlatformYear': prefix[0] if platform_code else None,
  })


df = df_nhtsa[['VIN', 'Make', 'Model', 'ModelYear']].copy()
df['CarName'] = df['Make'] + ' ' + df['Model'] + ' ' + df['ModelYear'].astype(str)
df.drop(columns=['Make', 'Model', 'ModelYear'], inplace=True)
df = df.join(df_nhtsa.apply(get_platform_code, axis=1))
df.head()

,VIN,CarName,PlatformCode,PlatformYear
0,3FTTW8F93NRA33503,FORD Maverick 2022,14D003-Z6C,N
1,1FMCU0G65LUB57012,FORD Escape 2020,14D003-X6C,L
2,MAJ6S3KL8LC349784,FORD Ecosport 2020,14C217-N15,G
3,1FT8W3DT7NEE61262,FORD F-350 2022,None,None
4,1FMJU1KT9MEA41589,FORD Expedition 2021,14D003-L3V,K


In [3]:
df_groups = df.groupby(['CarName', 'PlatformCode', 'PlatformYear'], dropna=False).size().reset_index(name='count')
df_groups.head()

,CarName,PlatformCode,PlatformYear,count
0,FORD Bronco 2021,14D003-B3C,M,99
1,FORD Bronco 2022,14D003-B3C,M,87
2,FORD Bronco 2022,14D003-B3C,N,118
3,FORD Bronco 2022,14D003-L38,N,6
4,FORD Bronco 2023,14D003-B3C,N,605


In [4]:
for group in df_groups.groupby(['PlatformCode', 'PlatformYear'], dropna=False):
  print(group[0])
  print(f'CarName: {group[1]["CarName"].unique()}')
  print()

('14C217-E81', 'A')
CarName: ['FORD Fiesta 2017' 'FORD Fiesta 2018' 'FORD Fiesta 2019']

('14C217-N15', 'G')
CarName: ['FORD Ecosport 2018' 'FORD Ecosport 2019' 'FORD Ecosport 2020'
 'FORD Ecosport 2021' 'FORD Ecosport 2022']

('14C217-V6T', 'C')
CarName: ['FORD Focus 2017' 'FORD Focus 2018' 'FORD Transit Connect 2018']

('14C217-V6T', 'H')
CarName: ['FORD C-Max 2018' 'FORD Escape 2018' 'FORD Escape 2019' 'FORD Focus 2018'
 'FORD Transit Connect 2018']

('14D003-1MC', 'L')
CarName: ['FORD Explorer 2020' 'FORD Explorer 2021' 'FORD Explorer 2022'
 'LINCOLN Aviator 2020' 'LINCOLN Aviator 2021' 'LINCOLN Aviator 2022']

('14D003-1MC', 'M')
CarName: ['FORD Explorer 2021' 'FORD Explorer 2022' 'LINCOLN Aviator 2020'
 'LINCOLN Aviator 2021' 'LINCOLN Aviator 2022' 'LINCOLN Aviator 2023']

('14D003-1MC', 'P')
CarName: ['FORD Explorer 2020' 'FORD Explorer 2022' 'FORD Explorer 2023'
 'FORD Explorer 2024' 'LINCOLN Aviator 2020' 'LINCOLN Aviator 2023'
 'LINCOLN Aviator 2024']

('14D003-2GC', 'K')
Car

In [5]:
from notebooks.ford.platforms import find_openpilot_platform


def get_openpilot_platform(row):
  return pd.Series({
    'CarInfoPlatform': find_openpilot_platform(row['CarName']),
  })


df_openpilot_platforms = df.join(df.apply(get_openpilot_platform, axis=1))
df_openpilot_platforms.head()

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /mnt/c/Users/camer/AppData/Local/Programs/Microsoft VS Code


,VIN,CarName,PlatformCode,PlatformYear,CarInfoPlatform
0,3FTTW8F93NRA33503,FORD Maverick 2022,14D003-Z6C,N,FORD MAVERICK 1ST GEN
1,1FMCU0G65LUB57012,FORD Escape 2020,14D003-X6C,L,FORD ESCAPE 4TH GEN
2,MAJ6S3KL8LC349784,FORD Ecosport 2020,14C217-N15,G,None
3,1FT8W3DT7NEE61262,FORD F-350 2022,None,None,None
4,1FMJU1KT9MEA41589,FORD Expedition 2021,14D003-L3V,K,None


In [6]:
df_openpilot_platforms['CarInfoPlatform'].value_counts(dropna=False)

CarInfoPlatform
None                            27023
FORD EXPLORER 6TH GEN            3332
FORD ESCAPE 4TH GEN              2164
FORD F-150 14TH GEN              1338
FORD BRONCO SPORT 1ST GEN         930
FORD MUSTANG MACH-E 1ST GEN       813
FORD MAVERICK 1ST GEN             788
FORD F-150 LIGHTNING 1ST GEN      374
Name: count, dtype: int64

In [7]:
for group in df_openpilot_platforms.groupby('CarInfoPlatform', dropna=False):
  print(group[0])
  print(f'CarName: {group[1]["CarName"].unique()}')
  print(f'PlatformCode: {group[1]["PlatformCode"].unique()}')
  print(f'PlatformYear: {group[1]["PlatformYear"].unique()}')
  print()

FORD BRONCO SPORT 1ST GEN
CarName: ['FORD Bronco Sport 2021' 'FORD Bronco Sport 2022']
PlatformCode: ['14D003-X6C']
PlatformYear: ['L']

FORD ESCAPE 4TH GEN
CarName: ['FORD Escape 2020' 'FORD Escape 2022' 'FORD Escape 2021']
PlatformCode: ['14D003-X6C']
PlatformYear: ['L']

FORD EXPLORER 6TH GEN
CarName: ['LINCOLN Aviator 2020' 'FORD Explorer 2022' 'FORD Explorer 2021'
 'FORD Explorer 2020' 'FORD Explorer 2023' 'LINCOLN Aviator 2021']
PlatformCode: ['14D003-1MC']
PlatformYear: ['L' 'M' 'P']

FORD F-150 14TH GEN
CarName: ['FORD F-150 2023']
PlatformCode: ['14D003-L3V']
PlatformYear: ['M']

FORD F-150 LIGHTNING 1ST GEN
CarName: ['FORD F-150 Lightning 2023' 'FORD F-150 Lightning 2022']
PlatformCode: ['14D003-L38']
PlatformYear: ['N' 'R']

FORD MAVERICK 1ST GEN
CarName: ['FORD Maverick 2022' 'FORD Maverick 2023']
PlatformCode: ['14D003-Z6C']
PlatformYear: ['N']

FORD MUSTANG MACH-E 1ST GEN
CarName: ['FORD Mustang Mach-E 2023' 'FORD Mustang Mach-E 2022'
 'FORD Mustang Mach-E 2021']
Platform

In [10]:
for group in df_openpilot_platforms.groupby(['PlatformCode', 'PlatformYear'], dropna=False):
  platforms = group[1]['CarInfoPlatform'].unique()
  if len(platforms) == 1 and platforms[0] is None:
    continue
  print(group[0])
  print(f'CarInfoPlatform: {group[1]["CarInfoPlatform"].unique()}')
  print(f'CarName: {group[1]["CarName"].unique()}')
  print()

('14D003-1MC', 'L')
CarInfoPlatform: [<CAR.EXPLORER_MK6: 'FORD EXPLORER 6TH GEN'> None]
CarName: ['LINCOLN Aviator 2020' 'FORD Explorer 2021' 'FORD Explorer 2020'
 'LINCOLN Aviator 2021' 'LINCOLN Aviator 2022' 'FORD Explorer 2022']

('14D003-1MC', 'M')
CarInfoPlatform: [<CAR.EXPLORER_MK6: 'FORD EXPLORER 6TH GEN'> None]
CarName: ['FORD Explorer 2022' 'FORD Explorer 2021' 'LINCOLN Aviator 2022'
 'LINCOLN Aviator 2023' 'LINCOLN Aviator 2021' 'LINCOLN Aviator 2020']

('14D003-1MC', 'P')
CarInfoPlatform: [None <CAR.EXPLORER_MK6: 'FORD EXPLORER 6TH GEN'>]
CarName: ['LINCOLN Aviator 2023' 'FORD Explorer 2023' 'LINCOLN Aviator 2024'
 'FORD Explorer 2024' 'FORD Explorer 2022' 'FORD Explorer 2020'
 'LINCOLN Aviator 2020']

('14D003-J9C', 'L')
CarInfoPlatform: [<CAR.MUSTANG_MACH_E_MK1: 'FORD MUSTANG MACH-E 1ST GEN'> None]
CarName: ['FORD Mustang Mach-E 2023' 'FORD Mustang Mach-E 2022'
 'FORD Mustang Mach-E 2024' 'FORD Mustang Mach-E 2021']

('14D003-L38', 'N')
CarInfoPlatform: [<CAR.F_150_LIGHTNI

In [9]:
PSCM_A = '14C217'
PSCM_B = '14D003'

def predict_platform(platform_code: str) -> str | None:
  if platform_code is None:
    return None

  core, prefix = platform_code.split('-')

  # Not recognised
  if core not in (PSCM_A, PSCM_B):
    return None

  # Not supported in openpilot
  if core == PSCM_A:
    return None

  raise ValueError(f'Unknown platform code: {platform_code}')